In [47]:
import os
import time
import subprocess
import paramiko

def remote_simulate(
        host="158.132.134.38",
        user="knpob",
        local_dat_folder="D:\\knpob\\20230613-FE cluster\\output\\e2x1",
        remote_dat_folder="D:\\knpob\\e2x1",
        dat_file="e2x1.dat",
    ):
    start_time = time.time()

    # setup ssh tunnel
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.load_system_host_keys()
    ssh.connect(hostname=host, username=user)

    def remote_exec(command, description, cwd=remote_dat_folder):
        stdin, stdout, stderr = ssh.exec_command(f'cd "{cwd}"; ' + command)
        exit_status = stdout.channel.recv_exit_status()
        current_time = time.time() - start_time

        if exit_status == 0:
            print(f'{user}@{host}:{remote_dat_folder}\n\033[32m{current_time:.2f}s: > successful > {description}\033[0m')
        else:
            print(f'{user}@{host}:{remote_dat_folder}\n\033[31m{current_time:.2f}s: > failed > {description}\033[0m')

    def local_exec(command, description, cwd=local_dat_folder):
        result = subprocess.run(command, shell=True, cwd=cwd)
        exit_status = result.returncode
        current_time = time.time() - start_time

        if exit_status == 0:
            print(f'{user}@{host}:{remote_dat_folder}\n\033[32m{current_time:.2f}s: > successful > {description}\033[0m')
        else:
            print(f'{user}@{host}:{remote_dat_folder}\n\033[31m{current_time:.2f}s: > failed > {description}\033[0m')

    # transfer dat file
    remote_exec(f'mkdir "{remote_dat_folder}"', description='create remote folder', cwd='~')
    local_exec(f'scp "{dat_file}" "{user}@{host}:{remote_dat_folder}"', description='load input file')
        
    # launch simulation task
    remote_exec(
        f'& "C:\\Program Files\\MSC.Software\\Marc\\2019.0.0\\marc2019\\tools\\run_marc.bat" -jid "{dat_file}" -back yes -nps 4 -nts 3 -nte 3 -nsolver 6', 
        description='finite element simulation')

    # simulation results feedback
    # p.s. in production, .t19 should be replaced with .t16
    local_result_folder = os.path.split(remote_dat_folder)[-1]
    local_exec(f'mkdir "{local_result_folder}"', description='create result folder')

    remote_result_path = os.path.join(remote_dat_folder, dat_file.replace('.dat', '.t19'))
    local_exec(f'scp "{user}@{host}:{remote_result_path}" "{local_result_folder}"', description='retrive simulation result')
    
    remote_sts_path = os.path.join(remote_dat_folder, dat_file.replace('.dat', '.sts'))
    local_exec(f'scp "{user}@{host}:{remote_sts_path}" "{local_result_folder}"', description='retrive simulation statistics')

In [48]:
import threading

def run_with_threads(n_jobs):
    start_time = time.time()
    threads = []

    for id in range(n_jobs):
        thread = threading.Thread(target=remote_simulate, kwargs={ 'remote_dat_folder': f"D:\\knpob\\e2x1_{id}" })
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    duration = time.time() - start_time
    print("-"*100)
    print(f"simulation tasks completed in {duration:.2f}s with {n_jobs} thread.")

In [49]:
run_with_threads(n_jobs=3)

knpob@158.132.134.38:D:\knpob\e2x1_2
0.49s: > successful > create remote folder
knpob@158.132.134.38:D:\knpob\e2x1_0
0.49s: > successful > create remote folder
knpob@158.132.134.38:D:\knpob\e2x1_1
0.49s: > successful > create remote folder
knpob@158.132.134.38:D:\knpob\e2x1_1
0.97s: > successful > load input file
knpob@158.132.134.38:D:\knpob\e2x1_2
0.97s: > successful > load input fileknpob@158.132.134.38:D:\knpob\e2x1_0
0.97s: > successful > load input file

knpob@158.132.134.38:D:\knpob\e2x1_0
5.75s: > successful > finite element simulation
knpob@158.132.134.38:D:\knpob\e2x1_2
5.77s: > successful > finite element simulation
knpob@158.132.134.38:D:\knpob\e2x1_0
5.77s: > successful > create result folder
knpob@158.132.134.38:D:\knpob\e2x1_2
5.78s: > successful > create result folder
knpob@158.132.134.38:D:\knpob\e2x1_0
6.22s: > successful > retrive simulation result
knpob@158.132.134.38:D:\knpob\e2x1_2
6.22s: > successful > retrive simulation result
knpob@158.132.134.38:D:\knpob\e2x1_